In [8]:
from pathlib import Path

import duckdb

In [9]:
con = duckdb.connect()

In [5]:
ds_paths = list(Path("../data/raw").glob("*.parquet"))

In [18]:
ds_paths

[PosixPath('../data/raw/news_categories.parquet'),
 PosixPath('../data/raw/recognasumm.parquet'),
 PosixPath('../data/raw/uci_categories.parquet')]

In [19]:
df = con.execute(f"SELECT * FROM read_parquet('{ds_paths[1]}')").df()

In [20]:
df

,index,Titulo,Subtitulo,Noticia,Categoria,Autor,Data,URL,Autor_corrigido,Sumario,__index_level_0__
0,52037,Ex-BBB Paula se emociona com novo procedimento...,Ex-sister compartilhou vídeo da preparação par...,A ex-BBB 23 Paula Freitas compartilhou em suas...,Entretenimento,Extra,09/05/2023 10h53,https://extra.globo.com/entretenimento/noticia...,Extra,Ex-BBB Paula se emociona com novo procedimento...,0
1,45191,Trump sinaliza que anúncio de candidatura par...,"Em entrevista à New York Magazine, ex-presiden...",Gabby Orrda CNN Talvez no sinal mais forte até...,Internacional,CNN,14/07/2022 às 16:51 | Atualizado 14/07/2022 à...,https://www.cnnbrasil.com.br/internacional/tru...,CNN,Trump sinaliza que anúncio de candidatura para...,1
2,74910,Auxílio Emergencial 2021: Caixa começa a pagar...,Pagamento começa nesta quinta para trabalhador...,A Caixa Econômica Federal (Caixa) começa a pag...,Economia,Por G1,17/06/2021 00h00,https://g1.globo.com/economia/auxilio-emergenc...,G1,Auxílio Emergencial 2021: Caixa começa a pagar...,2
3,120941,Netflix lança novos recursos para controle dos...,Plataforma vai permitir que usuários coloquem ...,A empresa também adicionou um recurso para pro...,Política,Por Reuters,07/04/2020 22h02,https://g1.globo.com/economia/tecnologia/notic...,Reuters,Netflix lança novos recursos para controle dos...,3
4,114965,Estudo estima que 5% da população da Espanha c...,"Na região da capital, 11% da população teve te...","""O estudo constata que 5% da população espanho...",Ciência e Tecnologia,Por France Presse,13/05/2020 21h12,https://g1.globo.com/mundo/noticia/2020/05/13/...,France Presse,Estudo estima que 5% da população da Espanha c...,4
...,...,...,...,...,...,...,...,...,...,...,...
135267,26352,Cúpula da Otan redefine geopolítica global; U...,Ouça a análise dos principais fatos da semana ...,"Camila Olivoda CNN Neste episódio, Roberto No...",Brasil,CNN,14/07/2023 às 14:46 | Atualizado 14/07/2023 à...,https://www.cnnbrasil.com.br/nacional/cupula-d...,CNN,Cúpula da Otan redefine geopolítica global; Uc...,27050
135268,93150,Órgão do Senado avalia que há espaço para redu...,"Para Instituição Fiscal Independente (IFI), ta...","A Instituição Fiscal Independente (IFI), órgão...",Economia,"Por Ana Paula Castro, TV Globo — Brasília",19/07/2023 18h36,,Ana Paula Castro,Órgão do Senado avalia que há espaço para redu...,27051
135269,46728,Levantamento mostra que ataques cibernéticos ...,País é o 2º na América Latina com mais ataques...,Ingrid Oliveirada CNN O Brasil registrou no pr...,Ciência e Tecnologia,CNN,19/08/2022 às 17:42 | Atualizado 22/08/2022 à...,https://www.cnnbrasil.com.br/tecnologia/levant...,CNN,Levantamento mostra que ataques cibernéticos n...,27052
135270,22490,Como seu nome pode afetar sua personalidade,A primeira informação que estranhos aprendem s...,Você já pode ter refletido sobre as diferentes...,Economia,BBC,8 julho 2021,https://www.bbc.com/portuguese/vert-fut-57564726,BBC,Como seu nome pode afetar sua personalidade. A...,27053


In [23]:
import re
from urllib.parse import urlparse
from collections import Counter
from statistics import mean

def skeleton(url):
    """
    Convert a URL path into a skeleton by replacing variable segments
    (pure numbers or long hex strings) with placeholders.
    """
    p = urlparse(url)
    parts = p.path.split('/')
    skeleton_parts = []
    for seg in parts:
        if not seg:
            continue
        # numeric segment
        if re.fullmatch(r"\d+", seg):
            skeleton_parts.append('{num}')
        # hex or uuid-like segment
        elif re.fullmatch(r"[0-9a-fA-F]{8,}", seg):
            skeleton_parts.append('{id}')
        else:
            skeleton_parts.append(seg)
    return '/' + '/'.join(skeleton_parts)

def get_skeleton_counts(url_list, top_n=None):
    """
    Return a Counter of skeleton patterns from the URL list.
    If top_n is provided, return only the most common top_n patterns.
    """
    sigs = Counter(skeleton(u) for u in url_list)
    if top_n:
        return sigs.most_common(top_n)
    return sigs

def get_regex_counts(url_list, rules=None):
    """
    Classify URLs according to regex rules. Returns a dict of counts.

    rules: dict of name->compiled regex. If None, uses defaults.
    """
    if rules is None:
        rules = {
            'date_based': re.compile(r"/\d{4}/\d{2}/\d{2}/"),
            'numeric_id': re.compile(r"/\d+($|/)"),
            'slug_only': re.compile(r"/[a-z0-9-]+\.(?:html?|php)$", re.IGNORECASE),
        }
    counts = {name: 0 for name in rules}
    counts['other'] = 0

    for u in url_list:
        path = urlparse(u).path
        matched = False
        for name, pat in rules.items():
            if pat.search(path):
                counts[name] += 1
                matched = True
                break
        if not matched:
            counts['other'] += 1
    return counts

def get_path_depths(url_list):
    """
    Compute a list of path depths (# of non-empty segments) for each URL.
    """
    depths = []
    for u in url_list:
        parts = [seg for seg in urlparse(u).path.split('/') if seg]
        depths.append(len(parts))
    return depths


def get_path_depths(url_list):
    """
    Compute a list of path depths (# of non-empty segments) for each URL.
    """
    depths = []
    for u in url_list:
        parts = [seg for seg in urlparse(u).path.split('/') if seg]
        depths.append(len(parts))
    return depths

def get_depth_stats(url_list):
    """
    Return detailed statistics on URL path segments:
      - depth: min, mean, max number of segments
      - other_segments: stats on count of segments excluding last
      - other_numeric_segments: stats on numeric-only segments excluding last
      - other_text_segments: stats on text segments excluding last
      - last_segment_contains_number_percent: percentage of URLs whose last segment contains a digit
    """
    depths = []
    other_counts = []
    other_num_counts = []
    other_text_counts = []
    last_contains = []

    for u in url_list:
        parts = [seg for seg in urlparse(u).path.split('/') if seg]
        depth = len(parts)
        depths.append(depth)

        if depth == 0:
            # no segments
            other_counts.append(0)
            other_num_counts.append(0)
            other_text_counts.append(0)
            last_contains.append(False)
            continue

        last = parts[-1]
        # Indicator if last segment contains any digit
        last_contains.append(bool(re.search(r"\d", last)))

        # Other segments = all but last
        others = parts[:-1]
        count_others = len(others)
        num_others = sum(1 for seg in others if re.fullmatch(r"\d+", seg))
        text_others = count_others - num_others

        other_counts.append(count_others)
        other_num_counts.append(num_others)
        other_text_counts.append(text_others)

    def stats(lst):
        if not lst:
            return {'min': 0, 'mean': 0, 'max': 0}
        return {'min': min(lst), 'mean': mean(lst), 'max': max(lst)}

    total = len(url_list)
    percent_last_number = (sum(last_contains) / total * 100) if total else 0

    return {
        'depth': stats(depths),
        'other_segments': stats(other_counts),
        'other_numeric_segments': stats(other_num_counts),
        'other_text_segments': stats(other_text_counts),
        'last_segment_contains_number_percent': percent_last_number,
    }

def analyze_urls(url_list, top_skeletons=10):
    """
    Perform a comprehensive URL structure audit:
      - Skeleton grouping (top patterns)
      - Regex-based classification counts
      - Path depth statistics
    Returns a dict with keys:
      'skeletons': list of (pattern, count)
      'regex_counts': dict of classification counts
      'depth_stats': dict of min/mean/max depths
    """
    return {
        'skeletons': get_skeleton_counts(url_list, top_n=top_skeletons),
        'regex_counts': get_regex_counts(url_list),
        'depth_stats': get_depth_stats(url_list),
    }

In [24]:
report = analyze_urls(df["URL"].tolist(), top_skeletons=10)
print("Top skeletons:", report['skeletons'])
print("Regex classification:", report['regex_counts'])
print("Depth stats:", report['depth_stats'])

Top skeletons: [('/', 27405), ('/economia/noticia/{num}/{num}/{num}/bovespa.ghtml', 134), ('/economia/noticia/{num}/{num}/{num}/dolar.ghtml', 131), ('/portuguese/{num}', 10), ('/politica/ultimas-noticias/{num}/{num}/{num}/stf-julgamento-juiz-das-garantias.htm', 4), ('/ultimas-noticias/agencia-estado/{num}/{num}/{num}/mexico-aprova-uso-da-vacina-abdala-de-cuba-contra-a-covid-19.htm', 4), ('/saude/ultimas-noticias/redacao/{num}/{num}/{num}/covid-19-coronavirus-casos-mortes-04-de-janeiro.htm', 4), ('/saude/ultimas-noticias/redacao/{num}/{num}/{num}/vacinacao-covid-19-coronavirus-26-de-dezembro.htm', 4), ('/saude/ultimas-noticias/redacao/{num}/{num}/{num}/covid-19-coronavirus-casos-mortes-28-de-novembro.htm', 4), ('/saude/ultimas-noticias/redacao/{num}/{num}/{num}/covid-19-coronavirus-casos-mortes-11-de-dezembro.htm', 3)]
Regex classification: {'date_based': 52823, 'numeric_id': 13106, 'slug_only': 7081, 'other': 62262}
Depth stats: {'depth': {'min': 0, 'mean': 3.616409900053226, 'max': 10

In [25]:
df = con.execute(f"SELECT * FROM read_parquet('{ds_paths[2]}')").df()

In [26]:
df

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071


In [27]:
report = analyze_urls(df["URL"].tolist(), top_skeletons=10)
print("Top skeletons:", report['skeletons'])
print("Regex classification:", report['regex_counts'])
print("Depth stats:", report['depth_stats'])

Top skeletons: [('/id/{num}', 1174), ('/usatoday/article/{num}', 978), ('/apps/pbcs.dll/article\\', 671), ('/index.php\\', 652), ('/video/default.aspx\\', 633), ('/business-news/article.aspx\\', 593), ('/\\', 529), ('/AllNews.asp\\', 521), ('/News.asp\\', 498), ('/article/{num}/NEWS/{num}', 478)]
Regex classification: {'date_based': 83928, 'numeric_id': 136906, 'slug_only': 41676, 'other': 159909}
Depth stats: {'depth': {'min': 1, 'mean': 3.41392314266167, 'max': 13}, 'other_segments': {'min': 0, 'mean': 2.41392314266167, 'max': 12}, 'other_numeric_segments': {'min': 0, 'mean': 1.0432745686155216, 'max': 5}, 'other_text_segments': {'min': 0, 'mean': 1.3706485740461485, 'max': 11}, 'last_segment_contains_number_percent': 55.903498658914486}


# Structure notes

News categories (Huffpost) format: /entry/slug+uid